In [1]:
import torch

if torch.cuda.is_available:
    device = "cuda"
else:
  device = "cpu"

print(device)

cuda


In [2]:
import torch
print(torch.cuda.is_available())

False


# down

In [132]:
from google.colab import drive
drive.mount("/content/myDrive")

Drive already mounted at /content/myDrive; to attempt to forcibly remount, call drive.mount("/content/myDrive", force_remount=True).


In [133]:
import pandas as pd

data1 = pd.read_csv("/content/myDrive/MyDrive/life/EXplore/мероприятия/Марафон \"Цифровой Петербург\"/FuzzySearch/данные/fias_dict.csv")
data2 = pd.read_csv("/content/myDrive/MyDrive/life/EXplore/мероприятия/Марафон \"Цифровой Петербург\"/FuzzySearch/данные/ojf_dict2.csv")

In [134]:
data = pd.concat([data1, data2], axis=0, ignore_index=True)
print(len(data1))
print(len(data2))
print(len(data))

3475306
3514963
6990269


# Обработка полей 1

In [35]:
data_shf0 = data.sample(frac=1, random_state=44).reset_index(drop=True)

In [ ]:
data_shf0.head(20)

In [96]:
data_shf = data_shf0[:10000]

In [ ]:
data_shf.head()

,id,address
0,987494,"Санкт-Петербург г, ул Савушкина, 25 литера А ,..."
1,1755391,"Санкт-Петербург г, ул. Варшавская, д. 21, лите..."
2,555432,"Санкт-Петербург г, ул Полярников, 15 литера А ..."
3,3255985,"Санкт-Петербург г, ул Чебышевская, 11 литера А..."
4,1111594,"Санкт-Петербург г, п. Парголово, тер. Торфяное..."


In [97]:
import re

data_shf = data_shf.apply(lambda row: f"{row['address']}, {row['id']}", axis=1)
data_shf = data_shf.apply(
    lambda row: re.sub(
        r'(?<!, )(лите|к\.|корпус|корп\.|стр|строен)',  # Ищем слова, перед которыми НЕТ ", "
        r', \1',  # Добавляем ", " перед найденным словом
        row
    ) # Только если data_shf - DataFrame, для Series убрать axis=1
)

In [38]:
print(data_shf.iloc[0])

Санкт-Петербург г, ул Савушкина, 25 , литера А , кв. 2, 987494


In [98]:
data_shf_fields = data_shf.str.split(',', expand=True)

In [79]:
data_shf_fields.head()

,0,1,2,3,4,5,6,7,8,9,10
0,Санкт-Петербург г,ул Савушкина,25,литера А,кв. 2,987494,None,None,None,None,None
1,Санкт-Петербург г,ул. Варшавская,д. 21,,литер А,кв. 75,1755391,None,None,None,None
2,Санкт-Петербург г,ул Полярников,15,литера А,кв. 82,555432,None,None,None,None,None
3,Санкт-Петербург г,ул Чебышевская,11,литера А,кв. 95,3255985,None,None,None,None,None
4,Санкт-Петербург г,п. Парголово,тер. Торфяное,ул. Парнасная,д. 1а,,литер А,кв. 44,1111594,None,None


In [99]:
data_shf_fields = data_shf_fields.fillna(0).astype(str)
data_shf_fields.head()

,0,1,2,3,4,5,6,7,8,9
0,Санкт-Петербург г,ул Савушкина,25,литера А,кв. 2,987494,0,0,0,0
1,Санкт-Петербург г,ул. Варшавская,д. 21,литер А,кв. 75,1755391,0,0,0,0
2,Санкт-Петербург г,ул Полярников,15,литера А,кв. 82,555432,0,0,0,0
3,Санкт-Петербург г,ул Чебышевская,11,литера А,кв. 95,3255985,0,0,0,0
4,Санкт-Петербург г,п. Парголово,тер. Торфяное,ул. Парнасная,д. 1а,литер А,кв. 44,1111594,0,0


In [ ]:
#@title test
import pandas as pd

# Пример DataFrame
data_shf_fields = pd.DataFrame({
    'A': ['0', '1', '0', '1', '0'],
    'B': ['1', '0', '0', '1', '0'],
    'C': ['0', '0', '0', '1', '1']
})

# Выводим исходный DataFrame
print("Исходный DataFrame:")
print(data_shf_fields)

# Применяем функцию
grouped_dfs = split_dataframe_by_zeros(data_shf_fields)

# Выводим результат
for count, group_df in grouped_dfs.items():
    print(f"\nГруппа с {count} нулевыми значениями:")
    print(group_df)

In [100]:
def split_dataframe_by_zeros(df):
    zero_counts = (df == '0').sum(axis=1)
    grouped_indices = {}
    for count in zero_counts.unique():
        grouped_indices[count] = zero_counts[zero_counts == count].index.tolist()

    return grouped_indices


# Применяем функцию
grouped_dfs = split_dataframe_by_zeros(data_shf_fields)

# Выводим результат
full = 0
uniqs = []
for count, group_df in grouped_dfs.items():
    print(f"Группа с {count} нулевыми значениями:")
    uniqs.append(group_df)
    print(len(group_df))
    full += len(group_df)
print(full)

Группа с 4 нулевыми значениями:
4452
Группа с 2 нулевыми значениями:
405
Группа с 3 нулевыми значениями:
4922
Группа с 5 нулевыми значениями:
165
Группа с 1 нулевыми значениями:
48
Группа с 6 нулевыми значениями:
7
Группа с 0 нулевыми значениями:
1
10000


## 1 обработка нулей

# обработка 2

In [236]:
data_2 = data
data_2 = data_2.apply(lambda row: f"{row['address']}, {row['id']}", axis=1)

In [244]:
def split(df):
  streets = []
  street = []
  row_prev = "qwrqwrqrqwrq"
  for index, value in df.items():
    fields = value.split(',')

    if fields[1] == row_prev:
      street.append(index)
    else:
      row_prev = fields[1]
      if street:
        streets.append(street)
      street = []
      street.append(index)


  #print(len(streets))
  return streets
#print(streets)

#for st in streets:
#  print(len(st))


In [239]:
streets = split(data_2)

19233


In [183]:
df1 = data_2[streets[1]]

In [ ]:

data_2[streets[14]]

In [240]:
# УБИРАЮ МЕЛКИЕ ГРУППЫ И ОСТАВЛЮ ТОЛЬКО БОЛЬШИЕ

loners = []
for st in streets:
  if len(st) < 20:
    loners.extend(st)


streets2 = [st for st in streets if len(st) >= 20]

In [ ]:
print(len(loners))
data_2.iloc[loners[800:820]]

In [230]:
print(len(streets))
print(len(streets2))

19233
10412


## обрабатываю большие группы

In [241]:
# снова разглаживаю индексы, чтобы потом снова также разделить

from itertools import chain
flattened = list(chain.from_iterable(streets2))
len(flattened)

6956357

In [242]:
data_2_unions = data_2.iloc[flattened]

In [245]:
streets_unions = split(data_2_unions)
print(len(streets_unions))

5633


In [247]:
print(len(streets_unions[0]))
print(len(streets_unions[1]))
print(len(streets_unions[2]))
print(len(streets_unions[3]))

709
146
884
26247


In [251]:
data_2.iloc[streets_unions[0]]

,0
0,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
1,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
2,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
3,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
4,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
...,...
711,"Санкт-Петербург г, ул Полоцкая, 2 литера А , п..."
712,"Санкт-Петербург г, ул Полоцкая, 2 литера А , к..."
713,"Санкт-Петербург г, ул Полоцкая, 2 литера А , к..."
714,"Санкт-Петербург г, ул Полоцкая, 2 литера А , к..."


In [252]:
data_2.iloc[716]


'Санкт-Петербург г, ш Фильтровское, 7 литера О , помещ. 1-Н, 5521'

In [253]:
# Получаю все улицы

streets_uniq = [uni[0] for uni in streets_unions]

In [254]:
data_2[streets_uniq]

,0
0,"Санкт-Петербург г, ул Полоцкая, 15 к. 2 литера..."
716,"Санкт-Петербург г, ш Фильтровское, 7 литера О ..."
862,"Санкт-Петербург г, пер Спасский, 3 литера Б , ..."
1749,"Санкт-Петербург г, пр-кт Просвещения, 30 к. 1 ..."
28060,"Санкт-Петербург г, ул Елизаветинская, 21 к. 2 ..."
...,...
6986476,"Санкт-Петербург г, ул. Шевченко, д. 17, литер ..."
6988009,"Санкт-Петербург г, ул.. Шкиперский проток, д. ..."
6988043,"Санкт-Петербург г, пр-кт. Комендантский, д. 30..."
6988310,"Санкт-Петербург г, наб. Реки Фонтанки, д. 32/1..."


In [257]:
def str(df):
  streets = []
  for row in df:
    fields = row.split(',')
    streets.append(fields[1])
  return streets



In [258]:
all_uniq_streets = str(data_2[streets_uniq])

In [259]:
print(len(all_uniq_streets))

5633


In [261]:
print(all_uniq_streets[-100:])


[' линия. 4-я В.О.', ' линия. 5-я В.О.', ' линия. 6-я В.О.', ' линия. 7-я В.О.', ' линия. 8-я В.О.', ' линия. 9-я В.О.', ' линия. Биржевая', ' линия. Кадетская В.О.', ' наб. Лейтенанта Шмидта', ' пер. Академический', ' пр-кт. Большой В.О.', ' наб. Морская', ' проезд. Адмиральский', ' ул. Капитанская', ' ул. Кораблестроителей', ' ул. Наличная', ' линия. 19-я В.О.', ' линия. 20-я В.О.', ' линия. 21-я В.О.', ' линия. 23-я В.О.', ' линия. 24-я В.О.', ' линия. 26-я В.О.', ' линия. 27-я В.О.', ' линия. Косая', ' пр-кт. Большой В.О.', ' пр-кт. Среднегаванский', ' пр-кт. Средний В.О.', ' ул. Вёсельная', ' ул. Гаванская', ' ул. Детская', ' ул. Канареечная', ' ул. Карташихина', ' ул. Наличная', ' ул. Опочинина', ' ул. Шевченко', ' ул.. Шкиперский проток', ' наб. Реки Смоленки', ' пер. Декабристов', ' пер. Каховского', ' пр-кт. КИМа', ' ул. Железноводская', ' ул. Наличная', ' ул. Одоевского', ' ул. Уральская', ' линия. 10-я В.О.', ' линия. 11-я В.О.', ' линия. 12-я В.О.', ' линия. 13-я В.О.', ' л

# label2id

In [101]:
label2id = {
    "O": 0,
    "B-улица": 1,
    "I-улица": 2,
    "B-дом": 3,
    "I-дом": 4,
    "B-корпус": 5,
    "I-корпус": 6,
    "B-литера": 7,
    "I-литера": 8,
    "B-квартира": 9,
    "I-квартира": 10,
}
id2label = {v: k for k, v in label2id.items()}

In [102]:
import re

def get_label1(text):
    text = str(text).lower().strip()

    if 'санкт' in text:
        return label2id["O"]

    elif re.search(r',', text) or "." in text:
        return label2id["O"]

    elif re.search(r'(пр|проспект|пер|ул\.?|улица|б-р|бул\.?|пл\.?)', text):
        return label2id["B-улица"]

    elif re.fullmatch(r'\d+', text) or re.fullmatch(r'\d+-\d+', text) or 'д.' in text:
        return label2id["B-дом"]

    elif re.search(r'кв|квартира|помещ', text):
        return label2id["B-квартира"]

    elif re.search(r'литер', text):
        return label2id["B-литера"]

    elif re.search(r'к?', text):
        return label2id["B-корпус"]



    else:
        return label2id["O"]  # неизвестное или пустое




In [103]:
df1 = data_shf_fields.iloc[uniqs[0]].astype(str)

In [104]:
df1

,0,1,2,3,4,5,6,7,8,9
0,Санкт-Петербург г,ул Савушкина,25,литера А,кв. 2,987494,0,0,0,0
1,Санкт-Петербург г,ул. Варшавская,д. 21,литер А,кв. 75,1755391,0,0,0,0
2,Санкт-Петербург г,ул Полярников,15,литера А,кв. 82,555432,0,0,0,0
3,Санкт-Петербург г,ул Чебышевская,11,литера А,кв. 95,3255985,0,0,0,0
5,Санкт-Петербург г,пр-кт. Аптекарский,д. 18,литер А,кв. 496-H,2207394,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...
9991,Санкт-Петербург г,ул. Зверинская,д. 20,литер А,кв. 22,2344809,0,0,0,0
9994,Санкт-Петербург г,пр-кт. Московский,д. 186,литер А,кв. 147*,1733900,0,0,0,0
9995,Санкт-Петербург г,ул 1-я Никитинская,30,литера А,кв. 69,1361608,0,0,0,0
9996,Санкт-Петербург г,пр-кт. Науки,д. 65,литер А,кв. 84,1398014,0,0,0,0


In [ ]:
print(df1.columns)
print(df1.columns[5:])

RangeIndex(start=0, stop=10, step=1)
RangeIndex(start=5, stop=10, step=1)


In [49]:
df1_1 = df1.drop(df1.columns[5:], axis=1)

In [50]:
row= df1_1.iloc[0]
row.tolist()
str(row)

'0    Санкт-Петербург г\n1         ул Савушкина\n2                  25 \n3            литера А \n4                кв. 2\nName: 0, dtype: object'

### парсинг

In [22]:
#@title func pars

import pandas as pd

def parse(data):
    rows_words = []
    rows_tags = []

    for row in data.iloc:
        row_words = []
        row_tags = []
        #print(row)
        #print(type(row))

        list0 = row.tolist()

        field_idx = 1
        for field in list0:
            words = [str(word) for word in field.split()]

            for i, word in enumerate(words):
              row_words.append(word)
              row_tags.append(field_idx if i == 0 else field_idx + 1)
            row_words.append(",")
            row_tags.append(0)
            field_idx += 2

        rows_words.append(row_words)
        rows_tags.append(row_tags)

    df_words = pd.DataFrame(rows_words)
    df_tags = pd.DataFrame(rows_tags)

    return df_words, df_tags

In [51]:
df1_1_par_words, df1_1_par_tags = parse(df1_1)
df1_1_par_words.head(20)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,Санкт-Петербург,г,",",ул,Савушкина,",",25,",",литера,А,",",кв.,2,",",None,None,None,None
1,Санкт-Петербург,г,",",ул,Полярников,",",15,",",литера,А,",",кв.,82,",",None,None,None,None
2,Санкт-Петербург,г,",",ул,Чебышевская,",",11,",",литера,А,",",кв.,95,",",None,None,None,None
3,Санкт-Петербург,г,",",пр-кт,Литейный,",",25,",",литера,А,",",помещ.,17-Н,",",None,None,None,None
4,Санкт-Петербург,г,",",проезд.,1-й,Предпортовый,",",д.,13,",",строение,1,",",кв.,49,",",None,None
5,Санкт-Петербург,г,",",пр-кт,Маршала,Блюхера,",",56,",",литера,А,",",кв.,81,",",None,None,None
6,Санкт-Петербург,г,",",ул,Даниила,Хармса,",",7,",",литера,А,",",кв.,304,",",None,None,None
7,Санкт-Петербург,г,",",ул..,Ультрамариновая,",",д.,10,",",строение,1,",",кв.,618,",",None,None,None
8,Санкт-Петербург,г,",",ул,Ткачей,",",50,",",литера,Щ,",",кв.,48,",",None,None,None,None
9,Санкт-Петербург,г,",",ул,Коммунаров,",",124,",",литера,А,",",кв.,1,",",None,None,None,None


# etc

In [ ]:
#@title 1
import pandas as pd

data_words = []
data_tags = []

for i in range(len(data[:200000])):
    value = str(data.iloc[i, 1])

    if "к." in value:
        pos = value.find("к.")
        value = value[:pos-1] + "," + value[pos-1:]

    if "литера" in value:
        pos = value.find("литера")
        value = value[:pos-1] + "," + value[pos-1:]

    parts = [part.strip() for part in value.split(",") if part.strip() != ""]

    words_row = []
    tags_row = []

    for part in parts:
        text = str(part).strip()

        words_field = text.split()
        if not words_field:
                continue

        if "к." in part:
            words_row.append("к")
            tags_row.append(5)
            words_row.append(".")
            tags_row.append(0)
            words_row.append(part[part.index("к.") + 2:])
            tags_row.append(6)
            words_row.extend(",")
            tags_row.append(0)
            continue

        elif "кв." in part:
            words_row.append("кв")
            tags_row.append(9)
            words_row.append(".")
            tags_row.append(0)
            words_row.append(part[part.index("кв.") + 3:])
            tags_row.append(10)
            continue

        elif "помещ." in part:
            words_row.append("помещ")
            tags_row.append(9)
            words_row.append(".")
            tags_row.append(0)
            words_row.append(part[part.index("помещ.") + 6:])
            tags_row.append(10)
            continue

        tg = get_label1(words_field[0])
        tags_field = [tg]

        for word in words_field[1:]:
            tags_field.append(tg + 1 if tg != 0 else 0)

        words_row.extend(words_field)
        tags_row.extend(tags_field)
        words_row.extend(",")
        tags_row.append(0)

    data_words.append(words_row)
    data_tags.append(tags_row)

# Финальные датафреймы
data_words_df = pd.DataFrame(data_words).fillna(0)
data_tags_df = pd.DataFrame(data_tags).fillna(0).astype(int)



In [ ]:
x = 2000

data_words_df.iloc[x:x+50]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2000,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,21,0
2001,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,317,0
2002,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,439,0
2003,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",помещ,.,30-Н,0
2004,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,527,0
2005,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,70,0
2006,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,49,0
2007,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,310,0
2008,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,136,0
2009,Санкт-Петербург,г,",",пр-кт,Просвещения,",",30,",",к,.,1,",",литера,А,",",кв,.,318,0


In [ ]:
data_tags_df.iloc[x:x+50]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18
2000,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2001,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2002,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2003,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2004,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2005,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2006,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2007,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2008,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0
2009,0,0,0,1,2,0,3,0,5,0,6,0,7,8,0,9,0,10,0


# export

In [ ]:
data_words_df = data_words_df.drop(data_words_df.columns[20:50], axis=1)
data_tags_df = data_tags_df.drop(data_tags_df.columns[20:50], axis=1)

In [ ]:
data_words_df.to_csv("0_orig_dswords_shf.csv")
data_tags_df.to_csv("0_orig_dstags_shf.csv")